In [19]:
import json
import pandas as pd

def json_to_csv(json_file):
    
    with open(json_file) as file:
        parsed_data = json.load(file)
    
    for issue in parsed_data['issues']:
        issue['line_number'] = issue['textRange']['startLine']
        issue['file_path'] = issue.pop('component')
        issue['description'] = issue.pop('message')
        impact = issue['impacts'][0]
        issue['software_quality'] = impact['softwareQuality']
        issue['severity_impact'] = impact['severity']
        del issue['impacts']
    
    df = pd.DataFrame(parsed_data['issues'])
    
    write_path = json_file.replace('.json', '.csv')
    
    df[['file_path','line_number','type','severity','description']].to_csv(write_path, index=False)

data_paths = [
    "sonarcloud_data/sonarcloud_ebean.json",
    "sonarcloud_data/sonarcloud_hibernate-orm.json",
    "sonarcloud_data/sonarcloud_openjpa.json",
    "sonarcloud_data/sonarcloud_ponyorm.json",
    "sonarcloud_data/sonarcloud_sqlalchemy.json",
]

for path in data_paths:
    json_to_csv(path)